# Using Python Get Data From Baidu API

In [ ]:
#coding=utf-8
import json
import geopandas
import matplotlib.pyplot as plt
import requests
from shapely.geometry import LineString
import time
import os
import pandas as pd

In [2]:
BASE_URL = 'https://restapi.amap.com/v3/traffic/status/rectangle?rectangle={loc}&key={k}&extensions=all'
keys = [
    '0f77fda9a61b3fa8ae32f99b2eabe534', 'ac644665b36b3ca24b4a7abe54661efb',
    'd718d4a3addce3439b3df02c0e8b599c','2b239cb466670c8379747913be5ba79e'
]
Loc_of_Shanghai = '121.473658,31.230378'
key_flag = 3
Lines = []
Names = []
Speeds = []
Status = []
Lenghts = []
Lcodes = []
Timestamps = []
set_code = []

In [3]:
class Grid(object):
    def __init__(self, loc_all):
        self.loc_all = loc_all
        self.lng_sw = float(self.loc_all.split(',')[0])
        self.lat_sw = float(self.loc_all.split(',')[1])
        self.max = 0.06

    def get_max(self):
        for i in range(1, 10):
            data = str(self.lng_sw) + "," + str(self.lat_sw) + ';' + str(
                self.lng_sw + i / 100) + ',' + str(self.lat_sw + i / 100)
            print(i)
            url = BASE_URL.format(loc=data, k=keys[key_flag])
            print(url)
            res = requests.get(url)
            datas = res.json()
            if datas['status'] != '1':
                print(
                    "the maxium of distance is",
                    i,
                )
                break

    def get_grid(self):
        grid = []
        lng = []
        lat = []
        for i in range(-5,3):
            lng.append(self.lng_sw + self.max * i)
            lat.append(self.lat_sw + self.max * i)
        for i in range(8):
            for j in range(8):
                data = str(lng[i])+','+str(lat[j])+';'+str(lng[i]+self.max)+','+str(lat[j]+self.max)
                grid.append(data)
        return grid

In [4]:
loc = Grid(Loc_of_Shanghai)
locs= loc.get_grid()


In [6]:
#coding=utf-8

def request_map_one_minute():
    Times = time.strftime('%Y_%m_%d %H:%M:%S',time.localtime())
    global Lines, Lenghts, Lcodes, locs, keys ,key_flag,set_code
#     all_datas = []
    for i in locs:
        url = BASE_URL.format(loc=i, k=keys[3])
#         print(url)
        res = requests.get(url)
        datas = res.json()
#         all_datas.append(datas)
        if datas['status'] == '1':
            for road in datas['trafficinfo']['roads']:
                dada =  road['lcodes'].split(',')
                if len(dada) == 1 :
                    if dada[0] in set_code:
                        continue
                    else:
                        polyline = road['polyline']
                        polylines = [
                            (float(y[0]), float(y[1])) for y in
                            [x.split(',') for x in [i for i in polyline.split(';')]]
                        ]
                        line = LineString(polylines)
                        Lines.append(line)
                        Names.append(road['name'])
                        Lenghts.append(line.length)
                        Lcodes.append(dada[0])
        else:
            if datas['infocode'] == '10003':
                key_flag = (key_flag + 1) % len(keys)
            if datas['infocode'] == '20003':
                continue
            else:
                print("Error of status",datas['infocode'])



def save_to_file(data):
    datas = geopandas.GeoDataFrame(data)
    datas = pd.concat([datas,Roads])
    datas = datas.drop_duplicates( subset =['Lcode'],
    keep='first',
    inplace=False)
    datas = datas.reset_index(drop=True)
    datas.to_file("Information_Of_Road",encoding='utf-8')

In [ ]:
def print_ts(message):
    print ("[%s] %s"%(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), message))

def run(interval, command):
    print_ts("Starting every %s seconds during 5:00 -- 22:00."%interval)
    global Lines,Lenghts,Lcodes,locs,key,key_flag,set_code
    while True:
        T_hour = time.localtime()[3]
        if T_hour >= 5 and T_hour <= 21:
            if time.localtime()[4] % 10 == 0 :
                Roads = geopandas.read_file("Information_Of_Road/",encoding='utf-8')
                set_code = list(Roads['Lcode'].values)
                datas = {'name':Names,'Lcode':Lcodes,'lenght':Lenghts,"geometry":Lines}
                save_to_file(datas)
                print_ts("Finish Save to file")
                Lines.clear()
                Names.clear()
                Lenghts.clear()
                Lcodes.clear()
            # sleep for the remaining seconds of interval
            # execute the command
            time_remaining = interval-time.time()%interval
            print_ts("Sleeping until %s (%s seconds)..."%((time.ctime(time.time()+time_remaining)), time_remaining))
            time.sleep(time_remaining)
            try :
                request_map_one_minute()
            except:
                print("Error")
            print_ts("Starting command.")
            print_ts("-"*100)
        else:
            time.sleep(time_remaining*command)
            
if __name__=="__main__":
    Roads = geopandas.read_file("Information_Of_Road/",encoding='utf-8')
    set_code = list(Roads['Lcode'].values)
    interval = 120
    command  = 100
    run(interval, command)

[2019-11-23 20:04:35] Starting every 120 seconds during 5:00 -- 22:00.
[2019-11-23 20:04:35] Sleeping until Sat Nov 23 20:06:00 2019 (84.41746997833252 seconds)...
[2019-11-23 20:06:08] Starting command.
[2019-11-23 20:06:08] ----------------------------------------------------------------------------------------------------
[2019-11-23 20:06:08] Sleeping until Sat Nov 23 20:08:00 2019 (111.94852304458618 seconds)...
[2019-11-23 20:08:08] Starting command.
[2019-11-23 20:08:08] ----------------------------------------------------------------------------------------------------
[2019-11-23 20:08:08] Sleeping until Sat Nov 23 20:10:00 2019 (111.75809597969055 seconds)...
[2019-11-23 20:10:08] Starting command.
[2019-11-23 20:10:08] ----------------------------------------------------------------------------------------------------


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



[2019-11-23 20:10:08] Finish Save to file
[2019-11-23 20:10:08] Sleeping until Sat Nov 23 20:12:00 2019 (111.16150712966919 seconds)...
[2019-11-23 20:12:08] Starting command.
[2019-11-23 20:12:08] ----------------------------------------------------------------------------------------------------
[2019-11-23 20:12:08] Sleeping until Sat Nov 23 20:14:00 2019 (111.40819191932678 seconds)...
[2019-11-23 20:14:09] Starting command.
[2019-11-23 20:14:09] ----------------------------------------------------------------------------------------------------
[2019-11-23 20:14:09] Sleeping until Sat Nov 23 20:16:00 2019 (110.87731504440308 seconds)...
[2019-11-23 20:16:08] Starting command.
[2019-11-23 20:16:08] ----------------------------------------------------------------------------------------------------
[2019-11-23 20:16:08] Sleeping until Sat Nov 23 20:18:00 2019 (111.79621291160583 seconds)...
[2019-11-23 20:18:08] Starting command.
[2019-11-23 20:18:08] -------------------------------

[2019-11-23 21:14:09] Starting command.
[2019-11-23 21:14:09] ----------------------------------------------------------------------------------------------------
[2019-11-23 21:14:09] Sleeping until Sat Nov 23 21:16:00 2019 (110.81066179275513 seconds)...
[2019-11-23 21:16:08] Starting command.
[2019-11-23 21:16:08] ----------------------------------------------------------------------------------------------------
[2019-11-23 21:16:08] Sleeping until Sat Nov 23 21:18:00 2019 (111.79361295700073 seconds)...
Error
[2019-11-23 21:18:04] Starting command.
[2019-11-23 21:18:04] ----------------------------------------------------------------------------------------------------
[2019-11-23 21:18:04] Sleeping until Sat Nov 23 21:20:00 2019 (115.27144503593445 seconds)...
[2019-11-23 21:20:09] Starting command.
[2019-11-23 21:20:09] ----------------------------------------------------------------------------------------------------
[2019-11-23 21:20:10] Finish Save to file
[2019-11-23 21:20:

[2019-11-24 07:34:09] Starting command.
[2019-11-24 07:34:09] ----------------------------------------------------------------------------------------------------
[2019-11-24 07:34:09] Sleeping until Sun Nov 24 07:36:00 2019 (110.803307056427 seconds)...
[2019-11-24 07:36:08] Starting command.
[2019-11-24 07:36:08] ----------------------------------------------------------------------------------------------------
[2019-11-24 07:36:08] Sleeping until Sun Nov 24 07:38:00 2019 (111.60745286941528 seconds)...
[2019-11-24 07:38:08] Starting command.
[2019-11-24 07:38:08] ----------------------------------------------------------------------------------------------------
[2019-11-24 07:38:08] Sleeping until Sun Nov 24 07:40:00 2019 (111.77845001220703 seconds)...
[2019-11-24 07:40:08] Starting command.
[2019-11-24 07:40:08] ----------------------------------------------------------------------------------------------------
[2019-11-24 07:40:08] Finish Save to file
[2019-11-24 07:40:08] Slee

[2019-11-24 08:36:08] Starting command.
[2019-11-24 08:36:08] ----------------------------------------------------------------------------------------------------
[2019-11-24 08:36:08] Sleeping until Sun Nov 24 08:38:00 2019 (111.84809398651123 seconds)...
[2019-11-24 08:38:08] Starting command.
[2019-11-24 08:38:08] ----------------------------------------------------------------------------------------------------
[2019-11-24 08:38:08] Sleeping until Sun Nov 24 08:40:00 2019 (111.94009184837341 seconds)...
[2019-11-24 08:40:08] Starting command.
[2019-11-24 08:40:08] ----------------------------------------------------------------------------------------------------
[2019-11-24 08:40:09] Finish Save to file
[2019-11-24 08:40:09] Sleeping until Sun Nov 24 08:42:00 2019 (110.61625504493713 seconds)...
[2019-11-24 08:42:08] Starting command.
[2019-11-24 08:42:08] ----------------------------------------------------------------------------------------------------
[2019-11-24 08:42:08] Sl

[2019-11-24 09:38:54] Starting command.
[2019-11-24 09:38:54] ----------------------------------------------------------------------------------------------------
[2019-11-24 09:38:54] Sleeping until Sun Nov 24 09:40:00 2019 (65.96988606452942 seconds)...
[2019-11-24 10:01:37] Starting command.
[2019-11-24 10:01:37] ----------------------------------------------------------------------------------------------------
[2019-11-24 10:01:37] Sleeping until Sun Nov 24 10:02:00 2019 (22.6391658782959 seconds)...
[2019-11-24 10:02:09] Starting command.
[2019-11-24 10:02:09] ----------------------------------------------------------------------------------------------------
[2019-11-24 10:02:09] Sleeping until Sun Nov 24 10:04:00 2019 (110.64685487747192 seconds)...
